In [1]:
!pip install wget | tail -n 1
!pip install "scikit-learn==1.3.2" | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

In [83]:

url = "https://us-south.ml.cloud.ibm.com"
apikey="_UZv0Z0oQ-x_WoUI3oX0wNr0kh9dhGExSUehWKfwgqNy"


In [84]:
credentials = {
    "url": url,
    "apikey": apikey
}


In [85]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

project_id

'018f9e4b-ad00-4ef9-9868-4a6107deb33f'

In [86]:
import  types
import pandas as pd
from botocore.client import Config
import ibm_boto3

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_UZv0Z0oQ-x_WoUI3oX0wNr0kh9dhGExSUehWKfwgqNy',  # Your secret API key
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'bucket-ls9d6ddjo2vu33a'
object_key = 'legal_text_dataset.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body'] # downloading data set from cloud object stroage 
if not hasattr(body, "iter"):
    body.iter = types.MethodType( iter, body)
data = pd.read_csv(body)
data.head(10)

,Unnamed: 0,ID,Phrase,Sentiment
0,0,1,Getting nowhere with surplusage,-1
1,1,2,But the Court nowhere suggested that it would ...,-1
2,2,3,Petitioners objection to shaving his beard cla...,-1
3,3,4,That result clashes with everything else,-1
4,4,5,the tolerable duration of police inquiries in ...,0
5,5,6,retrial be tolerable if the trial error could ...,0
6,6,7,I would be inclined to agree.,0
7,7,8,the trial court was inclined to accept the pro...,1
8,8,9,a plaintiff could overcome these hurdles where...,1
9,9,10,the procedural hurdles it could impose before ...,-1


In [87]:
label_map = {
    -1: "negative",
    0: "neutral",
    1: "positive"
}

In [88]:
data.value_counts(['Sentiment'])

Sentiment
-1           282
 1           172
 0           122
Name: count, dtype: int64

In [89]:
from sklearn.model_selection import train_test_split

data_train, data_test, y_train, y_test = train_test_split(data['Phrase'], 
                                                    data['Sentiment'],
                                                    test_size=0.3,
                                                    random_state=33, 
                                                    stratify=data['Sentiment'])
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)

In [90]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL', 'MERLINITE_7B', 'GRANITE_20B_CODE_INSTRUCT', 'GRANITE_34B_CODE_INSTRUCT', 'GRANITE_3B_CODE_INSTRUCT', 'GRANITE_7B_LAB', 'GRANITE_8B_CODE_INSTRUCT', 'LLAMA_3_70B_INSTRUCT', 'LLAMA_3_8B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01']


In [91]:
model_id = ModelTypes.FLAN_T5_XXL

In [92]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.REPETITION_PENALTY: 1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1 
}

In [95]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


In [96]:
model.get_details()

{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'

In [97]:
instruction="""Determine the sentiment of the sentense.
Use either 'positive', 'negative','neutral'.Use the provided examples as a template. """

In [98]:
zero_shot_inputs = [{"input": text} for text in data_test['Phrase']]
for i in range(2):
    print(f"The sentence example {i+1} is:\n\t {zero_shot_inputs[i]['input']}\n")

The sentence example 1 is:
	 The Court rejects the CCAs conclusion that Moore failed to make the requisite showings with respect to intellectual functioning

The sentence example 2 is:
	 He argues on appeal that had Defendants written truthful reports, or testified truthfully in deposition



In [99]:
data_train_and_labels=data_train.copy()
data_train_and_labels['Sentiment']=y_train

In [100]:
few_shot_example=[]
few_shot_examples=[]
for phrase,sentiment in data_train_and_labels.groupby('Sentiment').apply(lambda x: x.sample(2)).values:
    few_shot_example.append(f"\tsentence:\t{phrase}\n\tsentiment: {sentiment}\n")
few_shot_examples=[''.join(few_shot_example)]

In [101]:
few_shot_inputs_ = [{"input": text} for text in data_test['Phrase'].values]
for i in range(2):
    print(f"The sentence example {i+1} is:\n {few_shot_inputs_[i]['input']}\n")
    print(f"\tSentiment: {y_test[i]}\n")

The sentence example 1 is:
 The Court rejects the CCAs conclusion that Moore failed to make the requisite showings with respect to intellectual functioning

	Sentiment: -1

The sentence example 2 is:
 He argues on appeal that had Defendants written truthful reports, or testified truthfully in deposition

	Sentiment: -1



In [102]:
results = []
for inp in few_shot_inputs_[:2]:
    results.append(model.generate(" ".join([instruction+few_shot_examples[0], inp['input']]))["results"][0])

In [103]:
import json

print(json.dumps(results, indent=2))

[
  {
    "generated_text": "neutral",
    "generated_token_count": 1,
    "input_token_count": 207,
    "stop_reason": "max_tokens"
  },
  {
    "generated_text": "neutral",
    "generated_token_count": 1,
    "input_token_count": 207,
    "stop_reason": "max_tokens"
  }
]


In [104]:
y_true = [label_map[label] for label in y_test.values[:2]]
y_true

['negative', 'neutral']

In [105]:
y_pred = [result['generated_text'] for result in results]
y_pred

['neutral', 'neutral']

In [106]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred, y_true))

0.5
